Purpose: See if any of my core genes overlap with GWAS hits (from paper: https://academic-oup-com.proxy1.cl.msu.edu/gigascience/article/doi/10.1093/gigascience/giac080/6673780#369569254).<br>
Author: Anna Pardo<br>
Date initiated: Sept. 5, 2023

In [1]:
# import modules
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.multitest import fdrcorrection

In [2]:
# load GWAS table (supplementary table S4 from paper linked above)
gwas = pd.read_csv("../../data/SupplementalDataFileS4_AllUniqueGWASPeaks.csv",sep=",",header="infer")
gwas.head()

,peakName,PeakChromosome,PeakSNPPosition,MaximumRMIPScore,SNPsInPeak,PeakStart,PeakEnd,PeakLength,ClosestGeneToPeak,SecondClosestGeneToPeak,ThirdClosestGeneToPeak,NumberOfSignificantTraits,AllSignificantTraits,NumberOfSignificantPhenotypeGroups,PhenotypeGroups
0,1,3,"160,809,058",56,57,"160,559,294","160,989,691",430397,Zm00001d042317,Zm00001d042319,Zm00001d042315,16,Anthesis1_L;Anthesis4_H;Anthesis6_H;Anthesis7_...,2,FloweringTime;Vegetative
1,2,8,"136,012,710",57,24,"135,928,821","136,325,345",396524,Zm00001d010988,Zm00001d010987,Zm00001d010989,12,Anthesis1_L;Anthesis5_H;Anthesis6_H;Anthesis7_...,2,FloweringTime;Vegetative
2,3,7,"113,976,392",19,12,"112,979,061","113,976,392",997331,Zm00001d020433,Zm00001d020432,Zm00001d020434,5,BranchDensity_C;BranchNumber1_C;BranchZoneLeng...,1,Inflorescence
3,4,8,"155,019,919",17,10,"155,019,919","155,021,976",2057,Zm00001d011585,Zm00001d011584,Zm00001d011583,3,Anthesis4_H;Anthesis7_H;Anthesis_J,1,FloweringTime
4,5,6,"15,523,359",57,13,"14,881,429","16,102,546",1221117,Zm00001d035227,Zm00001d035228,Zm00001d035229,5,SMV10DAI_B;SMV14DAI_B;SMV21DAI_B;SMV7DAI_B;SMV...,1,Disease


In [3]:
# these gene IDs are from B73 V4
## make a list of genes to paste into Ensembl's ID History Converter
## or just write them to a file...but first make a non-overlapping set (set union of all these genes)
c = set(list(gwas["ClosestGeneToPeak"]))
sc = set(list(gwas["SecondClosestGeneToPeak"]))
tc = set(list(gwas["ThirdClosestGeneToPeak"]))
allc = set.union(*[c,sc,tc])

In [4]:
with open("../../data/v4_genes_from_Addie_GWAS_GigaScience.txt","w+") as outfile:
    for g in allc:
        outfile.write(g+"\n")

In [5]:
# load list of core gene IDs as both V5 and V4 genes
cg = pd.read_csv("../../data/All_coregenes_V5-to-V4.txt",sep="\t",header="infer")
cg.head()

,GeneID_V5,GeneID_V4,Second_GeneID_V4,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Zm00001eb000750,Zm00001d027320,NaN,NaN,NaN,NaN
1,Zm00001eb001060,Zm00001d027353,NaN,NaN,NaN,NaN
2,Zm00001eb001240,Zm00001d027371,NaN,NaN,NaN,NaN
3,Zm00001eb002180,Zm00001d027480,NaN,NaN,NaN,NaN
4,Zm00001eb002270,Zm00001d027488,NaN,NaN,NaN,NaN


In [6]:
if type(cg.iloc[0,2])==float:
    print("yes")

yes


In [7]:
# convert this into a dictionary format
cgdict = {}
for g in range(len(list(cg["GeneID_V5"]))):
    k = cg.iloc[g,0]
    v4list = []
    for i in range(1,6):
        if type(cg.iloc[g,i])==str:
            v4list.append(cg.iloc[g,i])
    cgdict[k]=v4list

In [8]:
# find the dataframe of peaks where the closest to peak is a core gene
c_v4 = []
sc_v4 = []
tc_v4 = []
cv5 = []
scv5 = []
tcv5 = []
for k in cgdict.keys():
    l = cgdict[k]
    for g in l:
        if g in list(gwas["ClosestGeneToPeak"].unique()):
            c_v4.append(g)
            cv5.append(k)
        if g in list(gwas["SecondClosestGeneToPeak"].unique()):
            sc_v4.append(g)
            scv5.append(k)
        if g in list(gwas["ThirdClosestGeneToPeak"].unique()):
            tc_v4.append(g)
            tcv5.append(k)

In [9]:
# make dataframes
cdf = pd.DataFrame(list(zip(cv5,c_v4)),columns=["Closest_V5","ClosestGeneToPeak"])
scdf = pd.DataFrame(list(zip(scv5,sc_v4)),columns=["SecondClosest_V5","SecondClosestGeneToPeak"])
tcdf = pd.DataFrame(list(zip(tcv5,tc_v4)),columns=["ThirdClosest_V5","ThirdClosestGeneToPeak"])

In [10]:
len(cdf.index)

30

In [11]:
len(scdf.index)

36

In [12]:
len(tcdf.index)

32

In [13]:
# merge with GWAS results to get the agronomic traits linked with the closest genes
gwas_cdf = cdf.merge(gwas)

In [14]:
gwas_cdf["ClosestCore"] = "Yes"
gwas_cdf.head()

,Closest_V5,ClosestGeneToPeak,peakName,PeakChromosome,PeakSNPPosition,MaximumRMIPScore,SNPsInPeak,PeakStart,PeakEnd,PeakLength,SecondClosestGeneToPeak,ThirdClosestGeneToPeak,NumberOfSignificantTraits,AllSignificantTraits,NumberOfSignificantPhenotypeGroups,PhenotypeGroups,ClosestCore
0,Zm00001eb000750,Zm00001d027320,290,1,"2,882,752",6,1,"2,882,752","2,882,752",0,Zm00001d027321,Zm00001d027319,1,BushelAcreEquivalent_J,1,Agronomic,Yes
1,Zm00001eb006310,Zm00001d027936,181,1,"17,966,847",7,1,"17,966,847","17,966,847",0,Zm00001d027934,Zm00001d027935,1,NodesWithBraceRoots_J,1,Root,Yes
2,Zm00001eb021040,Zm00001d029628,357,1,"80,602,342",5,1,"80,602,342","80,602,342",0,Zm00001d029629,Zm00001d029627,1,EarWidth_J,1,Inflorescence,Yes
3,Zm00001eb024820,Zm00001d030086,148,1,"104,530,524",17,1,"104,530,524","104,530,524",0,Zm00001d030087,Zm00001d030088,1,EarsPerPlant_J,1,Agronomic,Yes
4,Zm00001eb047280,Zm00001d032873,225,1,"241,078,399",5,1,"241,078,399","241,078,399",0,Zm00001d032874,Zm00001d032870,1,RootAngle_I,1,Root,Yes


In [15]:
# do the same for second- and third-closest genes
gwas_scdf = scdf.merge(gwas)
gwas_tcdf = tcdf.merge(gwas)

In [16]:
len(gwas_cdf["peakName"].unique())+len(gwas_scdf["peakName"].unique())+len(gwas_tcdf["peakName"].unique())

101

In [17]:
# how many unique peaks does this represent?
cp = list(gwas_cdf["peakName"].unique())
sp = list(gwas_scdf["peakName"].unique())
tp = list(gwas_tcdf["peakName"].unique())

In [18]:
len(set(cp+sp+tp))

97

In [19]:
# save this set as a dataframe
cg_gwas_hits_names = list(set(cp+sp+tp))
cg_gwas_hits = gwas[gwas["peakName"].isin(cg_gwas_hits_names)]

In [20]:
cg_gwas_hits.head()

,peakName,PeakChromosome,PeakSNPPosition,MaximumRMIPScore,SNPsInPeak,PeakStart,PeakEnd,PeakLength,ClosestGeneToPeak,SecondClosestGeneToPeak,ThirdClosestGeneToPeak,NumberOfSignificantTraits,AllSignificantTraits,NumberOfSignificantPhenotypeGroups,PhenotypeGroups
24,25,1,"280,994,311",13,3,"280,994,269","281,039,919",45650,Zm00001d034005,Zm00001d034006,Zm00001d034007,2,EarHeight_M;RootAngle2_O,2,Root;Vegetative
55,56,1,"297,580,977",12,2,"297,580,977","297,582,588",1611,Zm00001d034600,Zm00001d034601,Zm00001d034602,2,BranchNumber1_C;BranchZoneLength_J,1,Inflorescence
64,65,10,"149,988,757",10,2,"149,958,731","149,988,757",30026,Zm00001d026689,Zm00001d026687,Zm00001d026690,1,CobWeightGrams_J,1,Agronomic
85,86,3,"230,172,221",8,2,"230,171,635","230,172,221",586,Zm00001d044506,Zm00001d044507,Zm00001d044504,1,SAMVolume_M,1,Cellular/Biochemical
92,93,4,"190,410,017",26,6,"190,332,784","190,410,017",77233,Zm00001d052457,Zm00001d052458,Zm00001d052453,6,CrudeAsh_K;EarHeight_L;Ncombustion_K;Nkjeltec_...,2,SeedComposition;Vegetative


In [22]:
cg_gwas_hits["AllSignificantTraits"].unique()

array(['EarHeight_M;RootAngle2_O', 'BranchNumber1_C;BranchZoneLength_J',
       'CobWeightGrams_J', 'SAMVolume_M',
       'CrudeAsh_K;EarHeight_L;Ncombustion_K;Nkjeltec_K;PlantHeight_L;Protein_K',
       'SMV10DAI_B;SMV14DAI_B;SMVAUPDC_B', 'EarsPerPlant_J',
       'TillersPerPlant_J', 'BranchDensity_C', 'NodesWithBraceRoots_J',
       'BranchesPerTassel_J', 'RootAngle_I', 'RootDepth2_O', 'EarWidth_J',
       'Ncombustion_K;Nkjeltec_K', 'Protein_K', 'EarHeight_M',
       'BiomassYield_G', 'BushelAcreEquivalent_J', 'Fructose_K',
       'PH-EH_L', 'Nodes_M',
       'SMV10DAI_B;SMV14DAI_B;SMV21DAI_B;SMV28DAI_B',
       'TasselLengthError_P', 'PlantHeight_J', 'SMV7DAI_B', 'SMV28DAI_B',
       'senescence2_F', 'BranchNumber_P', 'KernelRowNumber_J',
       'LeafCuticularConductance7_H', 'SMVAUPDC_B',
       'SMV28DAI_B;SMV35DAI_B;SMVAUPDC_B', 'Anthesis_G',
       'KernelsPerRow_J', 'Starch_K', 'BraceRoot_N',
       'RootAngle1_O;RootAngle2_O', 'StalkDiamThick_N;peri_N',
       'LeafAreaIndex_

In [ ]:
gwas.head()

In [23]:
# start setup from scratch
# I need a list of all the genes, and I need to note whether they are core or not, and whether they are close to GWAS hits or not

# load tpm data with list of genes
tpm = pd.read_csv("../../data/rawtpm_bptreat_noPEG.tsv",sep="\t",header="infer")
tpm.head()

,Sample,BioProject,Treatment,Zm00001eb000010,Zm00001eb000020,Zm00001eb000050,Zm00001eb000060,Zm00001eb000070,Zm00001eb000080,Zm00001eb000100,...,Zm00001eb442810,Zm00001eb442820,Zm00001eb442840,Zm00001eb442850,Zm00001eb442870,Zm00001eb442890,Zm00001eb442910,Zm00001eb442960,Zm00001eb442980,Zm00001eb443030
0,SRR11933261,PRJNA637522,Drought,12.553818,2.321077,0.042520,12.932676,5.253755,11.105837,0.409268,...,0.171184,0.000000,0.0,0.0,0.0,0.309501,0.0,0.000000,0.0,0.000000
1,SRR11933272,PRJNA637522,Drought,16.255838,3.110372,0.405226,7.214039,1.902461,2.346186,0.170305,...,0.108052,0.127878,0.0,0.0,0.0,6.703281,0.0,0.000000,0.0,0.000000
2,SRR11933250,PRJNA637522,Drought,9.028815,2.984479,0.000000,3.092442,2.586555,16.186141,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.417565,0.0,0.254123,0.0,1.213349
3,SRR11933029,PRJNA637522,Control,8.201340,2.385748,0.000000,1.726808,1.926412,19.600487,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,1.370075
4,SRR11933040,PRJNA637522,Drought,10.371251,2.799099,0.000000,1.280629,3.771234,19.717683,0.143764,...,0.178304,0.012158,0.0,0.0,0.0,9.625225,0.0,0.000000,0.0,2.352959


In [24]:
# wrangle tpm data
ttpm = tpm.set_index("Sample").drop(["BioProject","Treatment"],axis=1).transpose().reset_index().rename(columns={"index":"GeneID"})
ttpm.head()

Sample,GeneID,SRR11933261,SRR11933272,SRR11933250,SRR11933029,SRR11933040,SRR11932822,SRR11932811,SRR11933230,SRR11932879,...,Ms71D3C,Ki3D1C,CML228D1D,CML333D3D,MO18WD3C,B73D3C,NC358D3C,P39D3D,M162WD3D,M162WD1D
0,Zm00001eb000010,12.553818,16.255838,9.028815,8.201340,10.371251,37.430009,39.925873,30.677016,23.393003,...,1.417104,1.923525,1.427602,9.580153,1.228100,2.966207,1.791556,4.286976,3.435711,3.498243
1,Zm00001eb000020,2.321077,3.110372,2.984479,2.385748,2.799099,27.508819,22.440680,24.648455,7.595576,...,0.000000,1.799671,0.000000,0.000000,1.925157,0.561768,0.176413,0.781353,0.379497,0.463832
2,Zm00001eb000050,0.042520,0.405226,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.304751,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Zm00001eb000060,12.932676,7.214039,3.092442,1.726808,1.280629,29.510498,22.148225,22.170584,14.727189,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Zm00001eb000070,5.253755,1.902461,2.586555,1.926412,3.771234,7.005587,7.590336,5.274585,2.177748,...,0.000000,0.451827,0.000000,1.018369,0.000000,0.000000,0.000000,0.000000,1.660372,0.748587


In [ ]:
cgdict.keys()

In [25]:
# isolate the gene IDs
gid = list(ttpm["GeneID"].unique())

In [26]:
# load the results from the ID History Converter from Ensembl Plants
idres = pd.read_csv("../../data/Results-Zea_mays_Tools_IDMapper_.csv",sep=",",header="infer")
idres.head()

,Requested ID,Matched ID(s),Releases
0,Zm00001d052837,Zm00001eb199410,110: Zm00001eb199410.1104: Zm00001eb199410.1
1,Zm00001d025912,Zm00001eb427160,110: Zm00001eb427160.1104: Zm00001eb427160.1
2,Zm00001d008485,Zm00001eb334630,110: Zm00001eb334630.1104: Zm00001eb334630.1
3,Zm00001d033568,Zm00001eb053580,110: Zm00001eb053580.1104: Zm00001eb053580.1
4,Zm00001d012957,Zm00001eb211650,110: Zm00001eb211650.1104: Zm00001eb211650.1


In [27]:
# covert to a dictionary with V5 as key and V4 as value
gwasdict = {}
for i in range(len(idres.index)):
    k = idres.iloc[i,1]
    gwasdict[k] = idres.iloc[i,0]

In [ ]:
# this and the next two cells are for testing association with all genes associated with GWAS hits - first, second, and third
## closest
iscore = []
isgwas = []
for g in gid:
    if g in list(cgdict.keys()):
        iscore.append("Y")
    else:
        iscore.append("N")
    if g in list(idres["Matched ID(s)"].unique()):
        isgwas.append("Y")
    else:
        isgwas.append("N")
        
# make dataframe
gdf = pd.DataFrame(list(zip(gid,iscore,isgwas)),columns=["GeneID","isCore","isGWAShit"])
gdf.head()

In [ ]:
# make a crosstab - contingency table
data = pd.crosstab(index=gdf["isGWAShit"],columns=gdf["isCore"])
data

In [ ]:
# run Fisher's exact test
odds_ratio, p_value = stats.fisher_exact(data)
print('odd ratio is : ' + str(odds_ratio))
print('p value is : ' + str(p_value))

In [30]:
gwas.columns

Index(['peakName', 'PeakChromosome', 'PeakSNPPosition', 'MaximumRMIPScore',
       'SNPsInPeak', 'PeakStart', 'PeakEnd', 'PeakLength', 'ClosestGeneToPeak',
       'SecondClosestGeneToPeak', 'ThirdClosestGeneToPeak',
       'NumberOfSignificantTraits', 'AllSignificantTraits',
       'NumberOfSignificantPhenotypeGroups', 'PhenotypeGroups'],
      dtype='object')

In [31]:
# test for association with first closest genes and core genes
iscore = []
isgwas = []
for g in gid:
    if g in list(cgdict.keys()):
        iscore.append("Y")
    else:
        iscore.append("N")
        
    if g in gwasdict.keys():
        gv4 = gwasdict[g]
        if gv4 in list(gwas["ClosestGeneToPeak"].unique()):
            isgwas.append("Y")
        else:
            isgwas.append("N")
    else:
        isgwas.append("N")
gdf = pd.DataFrame(list(zip(gid,iscore,isgwas)),columns=["GeneID","isCore","isFirstGWAS"])

In [32]:
# make a crosstab - contingency table
data = pd.crosstab(index=gdf["isFirstGWAS"],columns=gdf["isCore"])
data

isCore,N,Y
isFirstGWAS,,
N,36949,831
Y,1196,28


In [33]:
# run Fisher's exact test
odds_ratio, p_value = stats.fisher_exact(data)
print('odd ratio is : ' + str(odds_ratio))
print('p value is : ' + str(p_value))

odd ratio is : 1.0409467579456593
p value is : 0.842744972159088


## Testing for association with traits

In [ ]:
# get a list of the traits
